In [ ]:
from tensorflow.keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.reshape(-1, 28, 28, 1) / 255.0
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0

print("train_X shape : ", train_X.shape)
print("test_X shape : ", test_X.shape)

train_X shape :  (60000, 28, 28, 1)
test_X shape :  (10000, 28, 28, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D()) # pool_size=(2, 2)
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 5, 5, 64)          0

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filename = "model_ditis.h5"
checkpoint = ModelCheckpoint(filepath=filename, monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          callbacks=[checkpoint, early_stopping],
          batch_size=128, epochs=10)

Epoch 1/10
468/469 [============================>.] - ETA: 0s - loss: 0.3424 - accuracy: 0.8933
Epoch 1: val_accuracy improved from -inf to 0.97890, saving model to model_ditis.h5
469/469 [==============================] - 14s 29ms/step - loss: 0.3422 - accuracy: 0.8934 - val_loss: 0.0660 - val_accuracy: 0.9789
Epoch 2/10
  5/469 [..............................] - ETA: 12s - loss: 0.1608 - accuracy: 0.9563

C:\Users\KOSA1\anaconda3\envs\tf2.14\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - ETA: 0s - loss: 0.1086 - accuracy: 0.9671
Epoch 2: val_accuracy improved from 0.97890 to 0.98510, saving model to model_ditis.h5
469/469 [==============================] - 13s 29ms/step - loss: 0.1086 - accuracy: 0.9671 - val_loss: 0.0437 - val_accuracy: 0.9851
Epoch 3/10
468/469 [============================>.] - ETA: 0s - loss: 0.0817 - accuracy: 0.9762
Epoch 3: val_accuracy improved from 0.98510 to 0.98820, saving model to model_ditis.h5
469/469 [==============================] - 14s 29ms/step - loss: 0.0817 - accuracy: 0.9762 - val_loss: 0.0344 - val_accuracy: 0.9882
Epoch 4/10
468/469 [============================>.] - ETA: 0s - loss: 0.0684 - accuracy: 0.9795
Epoch 4: val_accuracy improved from 0.98820 to 0.99040, saving model to model_ditis.h5
469/469 [==============================] - 14s 30ms/step - loss: 0.0683 - accuracy: 0.9795 - val_loss: 0.0301 - val_accuracy: 0.9904
Epoch 5/10
469/469 [==============================] - ETA: 0s -

In [ ]:
model.evaluate(test_X, test_y)

313/313 [==============================] - 1s 3ms/step - loss: 0.0211 - accuracy: 0.9929


[0.021106384694576263, 0.992900013923645]

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if cap.isOpened() :
    while True :
        ret, img = cap.read()
        if ret :
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, bin_img = cv2.threshold(g_img, 110, 225, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            try :
                for contour in contours :
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 5 :
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        roi = cv2.resize(roi, dsize=(50, 50))
                        roi = cv2.resize(roi, dsize=(24, 24))
                        A = np.zeros((28, 28))
                        A[2:-2, 2:-2] = roi[:, :]
                        A = A.reshape(-1, 28, 28, 1)
                        num = np.argmax(model.predict(A))
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
            except Exception as e :
                print(e)

            cv2.imshow("Camera", bin_img)
            if cv2.waitKey(1)&0xFF == 27 :
                break # ESC
        else :
            print("No Frame")
            break
else :
    print("Camera not opened")

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 14ms/step
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

1/1 [==============================] - 0s 23ms/step
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

1/1 [==============================] - 0s 14ms/step
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

1/1 [===================